In [1]:
#!/usr/bin/env python
# coding: utf-8

import rosbag, os, matplotlib, pickle
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
from qsrlib.qsrlib import QSRlib, QSRlib_Request_Message
from qsrlib_io.world_trace import Object_State, World_Trace
from qsrlib.qsrlib import QSR_QTC_BC_Simplified
import numpy as np
import pandas as pd
import datetime as dt

os.chdir("/home/loz/QTC_Trajectory_HMMs/from_bags/")

In [2]:
# In[2]:

bagFolder = "/home/loz/QTC_Trajectory_HMMs/from_bags/HRSI_situation_QTC_rosbags/"
lab_csvs = [os.path.join(dp, f) for dp, dn, filenames in os.walk(bagFolder) for f in filenames if os.path.splitext(f)[1] == '.csv']

lab_csvs

['/home/loz/QTC_Trajectory_HMMs/from_bags/HRSI_situation_QTC_rosbags/pb/l/22.csv',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/HRSI_situation_QTC_rosbags/pb/l/16.csv',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/HRSI_situation_QTC_rosbags/pb/l/35.csv',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/HRSI_situation_QTC_rosbags/pb/l/14.csv',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/HRSI_situation_QTC_rosbags/pb/l/10.csv',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/HRSI_situation_QTC_rosbags/pb/l/27.csv',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/HRSI_situation_QTC_rosbags/pb/l/28.csv',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/HRSI_situation_QTC_rosbags/pb/l/33.csv',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/HRSI_situation_QTC_rosbags/pb/l/31.csv',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/HRSI_situation_QTC_rosbags/pb/l/7.csv',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/HRSI_situation_QTC_rosbags/pb/l/13.csv',
 '/home/loz/QTC_Trajectory_HMMs/from_bags/HRSI_situation_QTC_rosbags/pb/l/18.csv',
 '/ho

In [3]:
quantisation_factor = 0.003

r_positions = []
h_positions = []

r_state_seqs = []
h_state_seqs = []

qsrlib = QSRlib()

qtc_seqs = {}
for csv_path in lab_csvs:
#     print(csv_path)
    sit_code = csv_path[67:].replace("/", "_")[:-4]
#     print(sit_code)
    
    df = pd.read_csv(csv_path, index_col=0)
    
#     print(df)
    

    world = World_Trace()
    
    h_state_seq = []
    r_state_seq = []
    for index, row in df.iterrows():
        t = (pd.to_datetime(index) - dt.datetime(1970,1,1)).total_seconds()
        
        h_state_seq.append(Object_State(name="human", timestamp=t, x=row["human_pose_x"], y=row["human_pose_y"]))
        r_state_seq.append(Object_State(name="robot", timestamp=t, x=row["robot_pose_x"], y=row["robot_pose_y"]))
        
    world.add_object_state_series(h_state_seq)
    world.add_object_state_series(r_state_seq)

    # make a QSRlib request message
    dynamic_args = {"qtccs": {"no_collapse": False, "quantisation_factor": quantisation_factor,
                                            "validate": False, "qsrs_for": [("human", "robot")]}}

    qsrlib_request_message = QSRlib_Request_Message('qtccs', world, dynamic_args)

    # request your QSRs
    qsrlib_response_message = qsrlib.request_qsrs(req_msg=qsrlib_request_message)
    
    # Get QSR at each timestamp
    timestamps = qsrlib_response_message.qsrs.get_sorted_timestamps()
#     print(timestamps)
    qtc_seq = []
    for t in timestamps:
        for val in qsrlib_response_message.qsrs.trace[t].qsrs.values():
            qtc_seq.append(val.qsr['qtccs'].replace(",",""))
#     print qtc_seq
#     print ""
    
    qtc_seqs[sit_code] = qtc_seq

In [4]:
qtc_seqs

{'ot_hotl_1': ['-0-0',
  '-+-+',
  '-0-0',
  '-+-+',
  '-+-0',
  '-0-0',
  '-+-+',
  '-0-0',
  '+0-0',
  '-+-+',
  '0+-+',
  '+0-0',
  '+0-+',
  '+0-0',
  '+--+',
  '+0-+',
  '+0-0',
  '+--+',
  '+0-0',
  '+--+',
  '+0-0',
  '+--+',
  '+0-0',
  '+--+',
  '+0-0',
  '+--+',
  '+0-0',
  '+--+',
  '+0+0',
  '+0-0',
  '+--+',
  '+-++',
  '+0-0',
  '+--+',
  '+-0+',
  '+--+',
  '+0-0',
  '+-++',
  '+0-0',
  '+--+',
  '0-++',
  '+--+',
  '+0-0',
  '+--+',
  '+0-0',
  '+--+',
  '+--0',
  '+0-0',
  '+--+',
  '+0-0',
  '+--+',
  '+--0',
  '+0-0',
  '+--+',
  '+--0',
  '+--+',
  '+0-0',
  '+-++',
  '+--0',
  '+0-0'],
 'ot_hotl_10': ['-+-0',
  '-0-0',
  '-+-+',
  '-+-0',
  '-+-+',
  '-0-0',
  '-+-+',
  '-0-0',
  '00-0',
  '-+-+',
  '0+-+',
  '++-+',
  '+0-+',
  '+0-0',
  '+--+',
  '+0-0',
  '+--+',
  '+0-0',
  '+--+',
  '+0-0',
  '+--+',
  '+0-0',
  '+--+',
  '+0-0',
  '00-0',
  '+--+',
  '+0-0',
  '+--+',
  '+0-0',
  '-0-0',
  '+--+',
  '+--0',
  '-0-0',
  '+0-0',
  '+--+',
  '+--0',
  '+0-0',
  

In [5]:
with open("controlled_lab_qtc_seqs.pickle", "w") as f:
    pickle.dump(qtc_seqs, f)

In [6]:
with open("controlled_lab_qtc_seqs.pickle", "r") as f:
    print(pickle.load(f))

{'pb_l_28': ['----', '-0-0', '----', '-0-0', '----', '---0', '----', '-0-0', '----', '---0', '-000', '----', '---0', '-0-0', '----', '-0-0', '----', '--0-', '----', '-0-0', '----', '-0-0', '0-0-', '----', '-0--', '-0-0', '0---', '--0-', '----', '-0-0', '0-0-', '-0--', '-0-0', '00--', '00-0', '-0--', '+0--', '++--', '+0-0', '0+0-', '+0-0', '-+--', '++--', '+0-0', '++--', '+0-0', '++--', '-0-0', '0+--', '++--', '00-0', '++--', '+0-0', '++--', '++-0', '+0-0', '-+--', '++--', '+0-0', '++--', '+0-0', '++--', '++-0', '++--'], 'pb_r_21': ['--+0', '-0+0', '---0', '--00', '--+0', '-0+0', '--++', '--+0', '-0+0', '+--+', '--+0', '-0+0', '0-0+', '--+0', '--++', '--+0', '-0+0', '--++', '-0+0', '--++', '-0+0', '0-++', '--++', '--+0', '-0+0', '0-++', '--++', '---+', '--++', '-0+0', '+-0+', '--++', '-0+0', '0-++', '--++', '-0++', '--++', '-0++', '-0+0', '-0++', '00++', '+0++', '-+++', '++++', '+0+0', '0+0+', '++++', '+0+0', '-+-+', '++++', '+++0', '+0+0', '++++', '+++0', '+0+0', '++++', '+++0', '+0+0'